In [1]:
import json
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

Read data. Sort transactions, orders and payment methods into three separate datasets. Combine customer information and transactions into one dataframe, because we don't have customerId in transactions. Other data objects have ids that will let us combine them.

We are doing this because we want to end up with one flat table.

We will also load the data with the nested data to make sure we didn't make a parsing error.

In [2]:
data = []
transactions = []
payment_methods = []
orders = []
with open("customers_generated_100000_seed42.jsonl", "r") as f:
    for line in f:
        row = json.loads(line)
        data.append(row)
        for t in row["transactions"]:
            transactions.append({**row["customer"], **t})
        for pm in row["paymentMethods"]:
            payment_methods.append({**pm})
        for o in row["orders"]:
            orders.append({**o})

df_t = pd.DataFrame(transactions)
df_pm = pd.DataFrame(payment_methods)
df_o = pd.DataFrame(orders)
df = pd.json_normalize(data)
df.columns = df.columns.str.replace("customer.", "")

In [3]:
df["n_orders"] = df["orders"].apply(len)
df["n_paymentMethods"] = df["paymentMethods"].apply(len)
df["n_transactions"] = df["transactions"].apply(len)

In [4]:
int(df["n_transactions"].sum()) == df_t.shape[0]

True

In [5]:
int(df["n_orders"].sum()) == df_o.shape[0]

True

In [6]:
int(df["n_paymentMethods"].sum()) == df_pm.shape[0]

True

The total numbers of nested items matches the number of rows in the item dataframes.

In [7]:
df_t.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,loggedAt,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed
0,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01T12:03:01+00:00,l1c32oc6,cidkwn,t3w5uzbik,31,False
1,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01T12:11:23+00:00,uzhr5xff,myzycw,t3w5uzbik,64,False
2,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01T13:11:26+00:00,0t0pvn9e,0fn9xu,t3w5uzbik,39,False
3,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01T15:39:19+00:00,fgq5c34d,dom5ig,3gdppq0y9,18,True
4,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01T15:36:06+00:00,xtad1ior,dom5ig,3gdppq0y9,18,False


In [8]:
df_o.head()

,orderId,orderAmount,orderState,orderShippingAddress,loggedAt
0,cidkwn,31,failed,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01T12:02:00+00:00
1,myzycw,64,fulfilled,"000 Edward Crescent\nArnoldtown, MD 24766",2023-01-01T12:10:00+00:00
2,0fn9xu,39,fulfilled,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01T13:10:00+00:00
3,dom5ig,18,fulfilled,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01T15:36:00+00:00
4,7eedtj,21,fulfilled,USS Faulkner\nFPO AE 60416-0002,2023-01-01T16:57:00+00:00


In [9]:
df_pm.head()

,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,loggedAt
0,t3w5uzbik,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01T11:09:00+00:00
1,en93iiuc0,False,card,Mastercard,Grand Credit Corporation,2023-01-01T13:37:00+00:00
2,3gdppq0y9,False,card,Voyager,Bastion Banks,2023-01-01T15:21:00+00:00
3,ioykl1cq9,False,card,Diners Club / Carte Blanche,e,2023-01-01T16:20:00+00:00
4,9chj755nf,False,card,Voyager,Bulwark Trust Corp.,2023-01-01T16:23:00+00:00


Merge all data into one big flat table.

In [10]:
df_big = (
    df_t
    .merge(df_o, on="orderId", suffixes=("_t", "_o"))
    .merge(df_pm, on="paymentMethodId", suffixes=("", "_pm"))
)
df_big = df_big.rename(columns={"loggedAt": "loggedAt_pm"})

In [11]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371966 entries, 0 to 371965
Data columns (total 20 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   customerEmail                     371966 non-null  object
 1   customerPhone                     371966 non-null  object
 2   customerDevice                    371966 non-null  object
 3   customerIPAddress                 371966 non-null  object
 4   customerBillingAddress            371966 non-null  object
 5   loggedAt_t                        371966 non-null  object
 6   transactionId                     371966 non-null  object
 7   orderId                           371966 non-null  object
 8   paymentMethodId                   371966 non-null  object
 9   transactionAmount                 371966 non-null  int64 
 10  transactionFailed                 371966 non-null  bool  
 11  orderAmount                       371966 non-null  int64 
 12  or

Explicitly convert all dates to datetime.

In [12]:
df_big.loc[:, "loggedAt_t"] = pd.to_datetime(df_big["loggedAt_t"])
df_big.loc[:, "loggedAt_o"] = pd.to_datetime(df_big["loggedAt_o"])
df_big.loc[:, "loggedAt_pm"] = pd.to_datetime(df_big["loggedAt_pm"])

Sort by transaction date to simplify feature calculation. This way we will only need to look at previous rows when making calculations.

In [13]:
df_big = df_big.sort_values(by="loggedAt_t")

In [14]:
df_big.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,loggedAt_t,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,orderAmount,orderState,orderShippingAddress,loggedAt_o,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,loggedAt_pm
0,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:03:01+00:00,l1c32oc6,cidkwn,t3w5uzbik,31,False,31,failed,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:02:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00
1,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:11:23+00:00,uzhr5xff,myzycw,t3w5uzbik,64,False,64,fulfilled,"000 Edward Crescent\nArnoldtown, MD 24766",2023-01-01 12:10:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00
2,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 13:11:26+00:00,0t0pvn9e,0fn9xu,t3w5uzbik,39,False,39,fulfilled,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 13:10:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00
4,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:06+00:00,xtad1ior,dom5ig,3gdppq0y9,18,False,18,fulfilled,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:00+00:00,False,card,Voyager,Bastion Banks,2023-01-01 15:21:00+00:00
3,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:39:19+00:00,fgq5c34d,dom5ig,3gdppq0y9,18,True,18,fulfilled,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:00+00:00,False,card,Voyager,Bastion Banks,2023-01-01 15:21:00+00:00


Implement feature calculation logic.

- Convert `loggedAt_t` (transaction date) column and `paymentMethodIssuer` into numpy arrays for ✨efficiency✨.
- Iterate through each row once.
- For each row we will calculate the 24 hour window start based on the row transaction date.
- We have the id of the row that was the window's start in a previous iteration. We check if it still satisfies the condition, i.e. lies within 24 hours of a current row's transaction date. If not, we move one row up, until we reach the new 24 hour window start or the current row.
- Use a dictionary to maintain a "sliding window" of counts. It updates the counts for each issuer as transactions enter and leave the 24-hour window.
- Add the count of the current issuer from the dict to the array for the feature values.
- Add the feature row to the dataframe.

In [15]:
dates = df_big["loggedAt_t"].to_numpy()
issuers = df_big["paymentMethodIssuer"].to_numpy()
n_transactions_fast = np.zeros(len(df_big), dtype=int)
issuer_counts = defaultdict(int)
ws_idx = 0

for i in tqdm(range(len(df_big))):
    window_start = dates[i] - pd.Timedelta(hours=24)
    while ws_idx < i and dates[ws_idx] < window_start:
        issuer_to_remove = issuers[ws_idx]
        issuer_counts[issuer_to_remove] -= 1
        if issuer_counts[issuer_to_remove] == 0:
            del issuer_counts[issuer_to_remove]
        ws_idx += 1

    current_issuer = issuers[i]
    n_transactions_fast[i] = issuer_counts[current_issuer]

    issuer_counts[current_issuer] += 1

df_big["n_transactions_24h"] = n_transactions_fast

100%|██████████| 371966/371966 [00:01<00:00, 262442.27it/s]


In [16]:
df_big.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,loggedAt_t,transactionId,orderId,paymentMethodId,transactionAmount,...,orderAmount,orderState,orderShippingAddress,loggedAt_o,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,loggedAt_pm,n_transactions_24h
0,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:03:01+00:00,l1c32oc6,cidkwn,t3w5uzbik,31,...,31,failed,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:02:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00,0
1,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 12:11:23+00:00,uzhr5xff,myzycw,t3w5uzbik,64,...,64,fulfilled,"000 Edward Crescent\nArnoldtown, MD 24766",2023-01-01 12:10:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00,1
2,e0ifd0tvbd@example.com,338-555-9279,vuzzpqk51fpkh1dnmmjb,118.63.193.40,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 13:11:26+00:00,0t0pvn9e,0fn9xu,t3w5uzbik,39,...,39,fulfilled,"829 Ramirez Dam Suite 648\nGatesview, OR 55669...",2023-01-01 13:10:00+00:00,False,card,JCB 16 digit,His Majesty Bank Corp.,2023-01-01 11:09:00+00:00,2
4,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:06+00:00,xtad1ior,dom5ig,3gdppq0y9,18,...,18,fulfilled,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:00+00:00,False,card,Voyager,Bastion Banks,2023-01-01 15:21:00+00:00,0
3,ertj5pht0h@example.com,419-555-7537,9xpseimvihcwi64ciyhe,240.242.124.242,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:39:19+00:00,fgq5c34d,dom5ig,3gdppq0y9,18,...,18,fulfilled,"63978 Luis Ports\nYvonneville, NE 63736-2980",2023-01-01 15:36:00+00:00,False,card,Voyager,Bastion Banks,2023-01-01 15:21:00+00:00,1


### Implementation analysis:

- **Time Complexity**: O(n). Each row is visited exactly twice: once by the main loop index `i` and once by the window start index `ws_idx`. Dictionary lookups and updates are, on average, O(1). This guarantees linear time performance regardless of data density.
- **Memory Usage**: O(U), where U is the number of unique issuers. A single dictionary stores the counts, and its size is bounded by the number of unique issuers. This way we can be very memory efficient.
- **Strengths**: Code only performs a single pass over the data after an initial sort.
- **How to Scale**: For better performance that scales linearly (O(n)):
  - The production database can be partitioned on date, this way we only need to consider at most two partitions to determine 24 hour periods.
  - We can separate `paymentMethodIssuer` using sharding, this will remove the need to use a dictionary with issuer counts - we can calculate feature for each issuer separately.